In [1]:
import pandas as pd
from pycaret.regression import *

df = pd.read_csv("../../data/99_cleaned.csv")

df = df.drop(columns=["Unnamed: 0", "name"], errors='ignore')
df


,price,location,bedroom,bathroom,land_area,building_area
0,200.0,Surabaya,2,1,78,78
1,200.0,"Jambangan, Surabaya",2,1,72,70
2,200.0,Surabaya,3,1,152,150
3,200.0,Surabaya,2,1,78,75
4,235.0,Surabaya,2,1,69,65
...,...,...,...,...,...,...
6849,2500.0,Surabaya,4,4,150,200
6850,2500.0,"Sukolilo, Surabaya",4,2,147,200
6851,2500.0,Surabaya,3,2,240,300
6852,2500.0,Surabaya,2,1,105,125


In [2]:
df.duplicated().sum()

1549

In [3]:
df = df.drop_duplicates()

In [4]:
df.duplicated().sum()

0

In [5]:
# import ydata_profiling as pp

# profile = pp.ProfileReport(df)
# profile.to_file('pp_report/Report_99.html')

In [6]:
setup(
    df, 
    target='price',
    transform_target=True, 
    normalize=True, 
    normalize_method='minmax',
    remove_outliers=True, 
    outliers_threshold=0.05,
    categorical_features=['location'],
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,price
2,Target type,Regression
3,Original data shape,"(5305, 6)"
4,Transformed data shape,"(5119, 6)"
5,Transformed train set shape,"(3527, 6)"
6,Transformed test set shape,"(1592, 6)"
7,Numeric features,4
8,Categorical features,1
9,Rows with missing values,0.1%


In [7]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,275.6341,133051.0648,364.6021,0.6051,0.2903,0.2358,0.7780
lightgbm,Light Gradient Boosting Machine,277.5778,135620.8511,368.0325,0.5971,0.2936,0.2388,0.2450
gbr,Gradient Boosting Regressor,286.6305,139450.3672,373.1596,0.5861,0.2975,0.2463,0.0700
rf,Random Forest Regressor,280.3601,142268.9375,377.0759,0.5775,0.2995,0.2407,0.1500
xgboost,Extreme Gradient Boosting,279.5616,142813.9141,377.6589,0.5762,0.2982,0.2363,0.0650
et,Extra Trees Regressor,287.5198,150566.3405,387.8641,0.5529,0.3050,0.2437,0.1220
ada,AdaBoost Regressor,319.4658,161349.3128,401.5101,0.5212,0.3286,0.2864,0.0510
knn,K Neighbors Regressor,306.9785,162868.2234,403.3726,0.5163,0.3193,0.2642,0.0430
lasso,Lasso Regression,341.3770,223152.5401,469.4688,0.3396,0.3423,0.3036,0.2910
llar,Lasso Least Angle Regression,341.3773,223152.5326,469.4688,0.3396,0.3423,0.3036,0.0370


In [ ]:
tuned_model = tune_model(best, search_library='scikit-optimize', search_algorithm='bayesian')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,270.3334,127738.0535,357.4046,0.6181,0.2775,0.2238
1,281.8365,135094.6961,367.5523,0.5896,0.2856,0.2302
2,272.2526,131012.8347,361.9570,0.5964,0.2650,0.2105
3,282.4564,141263.5086,375.8504,0.5911,0.2995,0.2439
4,283.2966,141040.3010,375.5533,0.5807,0.3181,0.2646
5,273.4314,142570.4414,377.5850,0.6125,0.3017,0.2385
6,267.4293,133832.3842,365.8311,0.6162,0.3013,0.2464
7,272.1341,128967.7439,359.1208,0.6089,0.3018,0.2443
8,264.0402,117680.6399,343.0461,0.6189,0.2742,0.2226


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [9]:
holdout_pred = predict_model(tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,281.6046,138429.4272,372.0611,0.6191,0.2993,0.2483


In [10]:
new_data = df.copy()
new_data.drop('price', axis=1, inplace=True)
new_data.head()

,location,bedroom,bathroom,land_area,building_area
0,Surabaya,2,1,78,78
1,"Jambangan, Surabaya",2,1,72,70
2,Surabaya,3,1,152,150
3,Surabaya,2,1,78,75
4,Surabaya,2,1,69,65


In [11]:
predictions = predict_model(tuned_model, data = new_data)
predictions.head()

,location,bedroom,bathroom,land_area,building_area,prediction_label
0,Surabaya,2,1,78,78,838.562369
1,"Jambangan, Surabaya",2,1,72,70,752.089670
2,Surabaya,3,1,152,150,1252.654032
3,Surabaya,2,1,78,75,841.014326
4,Surabaya,2,1,69,65,658.323671


In [12]:
# save_model(tuned_model, '99-CB')